## Chatbot Logic

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -qqq transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [3]:
import os
import torch
import random
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from IPython.display import Markdown, display
from datasets import Dataset

In [4]:
try:
    os.chdir("/content/drive/MyDrive/skripsi")
    print("Directory changed")
except OSError:
    print("Error: Can't change the Current Working Directory")

Directory changed


In [5]:
# Constants
start_message = "==== Hello! I am Alex and I am your virtual friend. If you need a listening ear, I'm always here. To end the chat, input 'exit' in the chatbox. ===="

prevention_messages = ["Are you okay? How long have you been feeling this way?",
                       "That sounds so painful, and I appreciate you sharing that with me. How can I help?",
                       "I know things seem bleak now, but it can be hard to see possible solutions when you feel so overwhelmed.",
                       "I'm concerned about you because I care, and I want to offer support however I can. You can talk to me.",
                       "I'm always here if you feel like talking.",
                       "I'm always here to listen, but do you think a therapist could help a little more?",
                       "Have you thought about talking to a therapist?",
                       "You can withstand any storm and when you are too tired to stand, I will hold you up. You are never alone.",
                       "You know I’m always here for you.",
                       "You’re allowed to have bad days, but remember tomorrow is a brand new day.",
                       "You’ve got a place here on Earth for a reason.",
                       "It's okay to have such thoughts but if they become overwhelming, don't keep it to yourself. I am here for you.",
                       "Everything is a season, and right now you’re in winter. It’s dark and cold and you can’t find shelter, but one day it’ll be summer, and you’ll look back and be grateful you stuck it out through winter.",
                       "I know you are going through a lot and you’re scared, but you will never lose me.",
                       "I know it feels like a lot right now. It’s OK to feel that way.",
                       "Is there anything I can do to make this day go easier for you?"]

helpline_message = "In times of severe distress where you need to speak with someone immediately, these are suicide hotline services available for you. You will be speaking with volunteers or professionals who are trained to deal with suicide crisis. Samaritans of Singapore (SOS; 24 hours): 1800 221 4444 Mental Health Helpline (24 hours): 6389 2222 Singapore Association for Mental Health (SAMH) Helpline: 1800 283 7019"

### Functions

In [6]:
def printmd(string):
    display(Markdown(string))

In [7]:
# Untuk load tokenizer dan model generate response
def load_tokenizer_and_model(model="microsoft/DialoGPT-large"):
  tokenizer = AutoTokenizer.from_pretrained(model)
  model = AutoModelForCausalLM.from_pretrained(model)

  return tokenizer, model

In [8]:
# Untuk load tokenizer dan model sentiment analysis (detect suicidal intention)
def load_suicide_tokenizer_and_model(tokenizer="bert-base-uncased", model='./Models/bert/'):
  suicide_tokenizer = AutoTokenizer.from_pretrained(tokenizer)
  suicide_model = AutoModelForSequenceClassification.from_pretrained('./Models/bert/')

  return suicide_tokenizer, suicide_model

In [9]:
# Detecting Suicidal Intention
def check_intent(text):
  global suicide_tokenizer, suicide_model
  tokenised_text = suicide_tokenizer.encode_plus(text, return_tensors="pt")
  logits = suicide_model(**tokenised_text)[0]
  prediction = round(torch.softmax(logits, dim=1).tolist()[0][1])
  return prediction

In [10]:
# Generate Response
# Requires Editing
def generate_response(tokenizer, model, chat_round, chat_history_ids):

  # Encode user input and End-of-String (EOS) token
  new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

  # Append tokens to chat history
  bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim = -1) if chat_round > 0 else new_input_ids

  # Generate response given maximum chat length history of 1250 tokens
  chat_history_ids = model.generate(bot_input_ids, max_length=1250, pad_token_id=tokenizer.eos_token_id)

  # Print response based on intent
  if check_intent(user_input):
    printmd("*Alex:* {}".format(random.choice(prevention_messages)))
    printmd("{}".format(helpline_message))
  else:
    printmd("*Alex:* {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

  # Return the chat history ids
  return chat_history_ids

In [11]:
# Initialize chatbot tokenizer and model
tokenizer, model = load_tokenizer_and_model()

# Initialize chatbot history variable
chat_history_ids = None

# Initialise suicide detection tokenizer and model
suicide_tokenizer, suicide_model = load_suicide_tokenizer_and_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



## Serverside (FastAPI)



In [13]:
!pip install fastapi==0.103.2 nest-asyncio pyngrok uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 3.5 MB/s eta 0:00:00


In [14]:
messages = ["Are you okay? How long have you been feeling this way?",
                       "That sounds so painful, and I appreciate you sharing that with me. How can I help?",
                       "I know things seem bleak now, but it can be hard to see possible solutions when you feel so overwhelmed.",
                       "I'm concerned about you because I care, and I want to offer support however I can. You can talk to me.",
                       "I'm always here if you feel like talking.",
                       "I'm always here to listen, but do you think a therapist could help a little more?",
                       "Have you thought about talking to a therapist?",
                       "You can withstand any storm and when you are too tired to stand, I will hold you up. You are never alone.",
                       "You know I’m always here for you.",
                       "You’re allowed to have bad days, but remember tomorrow is a brand new day.",
                       "You’ve got a place here on Earth for a reason.",
                       "It's okay to have such thoughts but if they become overwhelming, don't keep it to yourself. I am here for you.",
                       "Everything is a season, and right now you’re in winter. It’s dark and cold and you can’t find shelter, but one day it’ll be summer, and you’ll look back and be grateful you stuck it out through winter.",
                       "I know you are going through a lot and you’re scared, but you will never lose me.",
                       "I know it feels like a lot right now. It’s OK to feel that way.",
                       "Is there anything I can do to make this day go easier for you?"]



In [15]:
!pip install -q -U google-generativeai

In [16]:
import pathlib
import textwrap

import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-pro')

In [17]:
from fastapi import FastAPI, Body
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
import random
import nest_asyncio
from pyngrok import ngrok, conf
import uvicorn

chat_history = {}

class Message(BaseModel):
    userMessage: str
    userId: str

class EncourageRequest(BaseModel):
    userId: str

class Response(BaseModel):
    message: str
    sentiment: bool

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

therapist_prompt = """
Act as a compassionate and understanding therapist. Respond to the following user message with empathy, understanding, and provide helpful insights or guidance. Ensure your response is supportive and encouraging.
Keep message short, no longer than 50 words.

User Message: "{user_message}"

Therapist Response:
"""

def generate_content(user_message):
    complete_prompt = therapist_prompt.format(user_message=user_message)
    response = model.generate_content(complete_prompt)
    return response

@app.post('/chat/send/')
async def send_chat(message: Message):
    user_id = message.userId
    user_message = message.userMessage

    if user_id not in chat_history:
        chat_history[user_id] = []

    last_message = chat_history[user_id][-1] if chat_history[user_id] else ""

    chat_history[user_id].append(user_message)

    bot_message = generate_content(user_message)
    check_result = check_intent(last_message)

    cleaned_response = bot_message.text.replace('\n', '').strip()

    response_object = Response(
        message= random.choice(messages) if check_result else cleaned_response,
        sentiment=check_result
    )

    response = {
        "message": response_object.message,
        "sentiment": response_object.sentiment
    }

    return response

encourage_prompt = """
You are a compassionate and understanding therapist. Summarize the following chat history with empathy, understanding, and provide a message of encouragement to the user. Ensure your response is supportive and encouraging.
No longer than 50 words.

Chat History:
{chat_history}

Encouraging Summary:
"""

def generate_encouragement(chat_history):
    complete_prompt = encourage_prompt.format(chat_history=chat_history)
    response = model.generate_content(complete_prompt)
    cleaned_response = response.text.replace('\n', '').strip()
    return cleaned_response

@app.post('/chat/encourage/')
async def encourage_user(request: EncourageRequest):
    user_id = request.userId

    if user_id not in chat_history or not chat_history[user_id]:
        return {"message": "No chat history found for the user."}

    user_chat_history = "\n".join(chat_history[user_id])
    encouragement_message = generate_encouragement(user_chat_history)

    chat_history[user_id] = []

    response = {
        "message": encouragement_message
    }

    return response

# Ngrok
NGROK_API_KEY = "your_ngrok_api_key"  # replace with actual NGROK_API_KEY
conf.get_default().config_path = "/content/drive/MyDrive/skripsi/ngrok.yml"
ngrok_tunnel = ngrok.connect(name="safespace")
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=80)

INFO:     Started server process [1149]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:80 (Press CTRL+C to quit)


Public URL: https://540c-34-148-137-8.ngrok-free.app
INFO:     2001:448a:7030:8374:b456:7531:df20:ffe2:0 - "POST /chat/send/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:8374:b456:7531:df20:ffe2:0 - "POST /chat/encourage/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:8374:b456:7531:df20:ffe2:0 - "POST /chat/send/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:8374:b456:7531:df20:ffe2:0 - "POST /chat/send/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:8374:b456:7531:df20:ffe2:0 - "POST /chat/send/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:8374:b456:7531:df20:ffe2:0 - "POST /chat/send/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:8374:b456:7531:df20:ffe2:0 - "POST /chat/send/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:8374:b456:7531:df20:ffe2:0 - "POST /chat/send/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:8374:b456:7531:df20:ffe2:0 - "POST /chat/send/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:8374:b456:7531:df20:ffe2:0 - "POST /chat/encourage/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:8374:b456:7531:df20:ff

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1149]
